In [ ]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt

In [ ]:
data= pd.read_csv('../pydata-book-master/ch08/Haiti.csv')
data.info()

In [ ]:
data.loc[:,['INCIDENT DATE','LATITUDE','LONGITUDE']].iloc[:10,:]

In [ ]:
#pandas series s.iloc[]
data.loc[:,'CATEGORY'].iloc[:6]

In [ ]:
#pandas dataframe d.describe()
data.describe()

In [ ]:
# pandas series s.notna()  
# pandas series s.lt()
# pandas series s.gt()
data =data.loc[(data.loc[:,'LATITUDE'].gt(18)) & (data.loc[:,'LATITUDE'].lt(20) ) & 
               (data.loc[:,'LONGITUDE'].gt(-75)) & (data.loc[:,'LONGITUDE'].lt(-70)) &
              data.loc[:,'CATEGORY'].notna()]
data.describe()

In [ ]:
# 分步骤
# python Built-in Function set()
# python Sequence Types — list, tuple, range s.append()
# python set set.union()
# python Built-in Functions sorted()
# python str s.split()
# python Comparisons in and not in
# python str s.strip()
cat = '2. Urgences logistiques | Vital Lines'
code,names = cat.split('.')
print('|' in names)
names =names.split('|')[1]
print((code,names.strip()))
cat_series = data.loc[:,'CATEGORY']
catstr = cat_series.iloc[0]
stripped=[]
stripped.append(catstr.split(',')[0].strip())
print (stripped)
stripped.append(catstr.split(',')[1].strip())
print (stripped)
print(catstr.split(',')[2].strip()!='')
print (set(stripped))
catstr1 = cat_series.iloc[1]
stripped1=[]
stripped1.append(catstr1.split(',')[0].strip())
print (stripped1)
print(catstr1.split(',')[1].strip()!='')
print (set(stripped1))
print((set(stripped),set(stripped1)))

cats = sorted(set.union(*(set(stripped),set(stripped1))))
cat1=cats[0]
cat2=cats[1]
print(cats)
code,names = cat1.split('.')
names =names.split('|')[1]
dic =dict()
dic[code]=names
code,names = cat2.split('.')
dic[code]=names
print(dic)

In [ ]:
def to_cat_list(catstr):
    stripped=(x.strip() for x in catstr.split(','))
    return [x for x in stripped if x]

def get_all_categories(cat_series):
    cat_sets = (set(to_cat_list(x)) for x in cat_series)
    return sorted(set.union(*cat_sets))

def get_english(cat):
    code,names = cat.split('.')
    if '|' in names:
        names = names.split('|')[1]
    return code,names.strip()
all_cats = get_all_categories(data.loc[:,'CATEGORY'])
english_mapping = dict(get_english(x) for x in all_cats)
print(english_mapping['2a'])
print(english_mapping['6c'])

In [ ]:
# numpy Array manipulation routines np.unique()
# pandas Index pd.Index  
# numpy Array creation routines np.zeros()
# python Built-in Functions len()
# pandas dataframe d.index
# python IndexError IndexError: single positional indexer is out-of-bounds
# pandas dataframe ValueError: columns overlap but no suffix specified
# python Built-in Functions zip()
# python The break statement break

def get_code(seq):
    return [x.split('.')[0] for x in seq if x]
all_codes = get_code(all_cats)
code_index = pd.Index(np.unique(all_codes))
dummy_frame = pd.DataFrame(np.zeros((len(data),len(code_index))),index=data.index,
                          columns=code_index)

print(dummy_frame.iloc[:,:6].info())

for row,cat in zip(data.index,data.loc[:,'CATEGORY']):
    if row >= len(data.index):
        break
    codes = get_code(to_cat_list(cat))
    dummy_frame.loc[:,codes].iloc[row]=1
    
data=data.join(dummy_frame.add_prefix('category_'))
data.iloc[:,10:15].info()

def basic_haiti_map(ax=None,lllat=17.25,urlat=20.25,
                    lllon=-75,urlon=-71):
    # 创建极球面投影的Basemap实例
    m = Basemap(ax=ax,projection='stere',
               lon_0=(urlon + lllon)/2,
               lat_0=(urlat + lllat)/2,
               llcrnrlat=lllat,urcrnrlat=urlat,
               llcrnrlon=lllon,urcrnrlon=urlon,
               resolution='f')
    # 绘制海岸线、州界、国界以及地图边界
    m.drawcoastlines()
    m.drawstates()
    m.drawcountries()
    return m
fig, axes = plt.subplots(nrow=2,ncols=2,figsize=(12,10))
fig.subplots_adjust(hspace=0.05,wspace=0.05)

to_plot=['2a','1','3c','7a']

lllat=17.25;urlat=20.25;lllon=-75;urlon=-71

for code,ax in zip(to_plot,axes.flat):
    m = basic_haiti_map(ax,lllat=lllat,urlat=urlat,
                       lllon=lllon,urlon=urlon)
    cat_data = data[data['category_%s' % code] ==1]
    # 计算地图的投影坐标
    x,y =m(cat_data.LONGITUDE.values,cat_data.LATITUDE.values)
    
    m.plot(x,y,'k.',alpha=0.5)
    ax.set_title('%s: %s' % (code,english_mapping[code]))
    
    shapefile_path='../pydata-book-master/ch08/PortAuPrince_Roads/PortAuPrince_Roads'
    m.readshapefile(shapefile_path,'roads')

# Python图形化工具生态系统
## Chaco
![avater](8-26.png)
## mayavi
## 其他库
PyQwt, Veusz, gunplot-py, biggles
## 图形化工具的未来
基于Web技术的图形化是必然的发展趋势，非Web式的图形化开发工作近几年减慢了许多  
python以及其他数据分析和统计计算环境R都是如此  
开发方向就变成了实现数据分析和准备工具与Web浏览器之间更为紧密的集成